In [ ]:
import torch
from transformers import T5ForSequenceClassification, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
dataset = load_dataset('causes_discription.csv')

# Initialize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForSequenceClassification.from_pretrained('t5-base', num_labels=len(dataset['train'].features['label'].names))

# Prepare input data
def preprocess_function(examples):
    prefix = "classify text: "
    texts = [prefix + text for text in examples['text']]
    return tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

# Tokenize datasets
tokenized_train = dataset['train'].map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
tokenized_val = dataset['test'].map(preprocess_function, batched=True, remove_columns=dataset['test'].column_names)

# Set labels
tokenized_train = tokenized_train.add_column('labels', dataset['train']['label'])
tokenized_val = tokenized_val.add_column('labels', dataset['test']['label'])

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    load_best_model_at_end=True
)

# Metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

# Train model
train_results = trainer.train()

# Evaluate model
eval_results = trainer.evaluate()

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(train_results.history['train_loss'])
plt.title('Training Loss Over Time')
plt.xlabel('Training Steps')
plt.ylabel('Loss')
plt.savefig('training_loss.png')
plt.close()

# Get predictions for confusion matrix
predictions = trainer.predict(tokenized_val)
preds = np.argmax(predictions.predictions, axis=1)

# Plot confusion matrix
cm = confusion_matrix(predictions.label_ids, preds)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix.png')
plt.close()

# Print final accuracy
print(f"Final Validation Accuracy: {eval_results['eval_accuracy']:.4f}")